In [217]:
import pandas as pd
import numpy as np
import random
from functools import reduce
import math
from operator import add, itemgetter
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestRegressor

In [185]:
'''Import Data must be in the following format
  Name, Ref1, Ref2, Ref3, Team, Years, Pos, Wt, Last_Game_TOV,
  AverageTOV, Last_Game_REB, AverageREB, Last_Game_3PM, Average3PM,
  Last_Game_FGM, AverageFGM, Last_Game_PTS, AveragePTS, Last_Game_AST,
  AverageAST, Last_Game_STL, AverageSTL, Last_Game_BLK, AverageBLK,
  game_number, Home/Away, Opponent'''

'Import Data must be in the following format\n  Name, Ref1, Ref2, Ref3, Team, Years, Pos, Wt, Last_Game_TOV,\n  AverageTOV, Last_Game_REB, AverageREB, Last_Game_3PM, Average3PM,\n  Last_Game_FGM, AverageFGM, Last_Game_PTS, AveragePTS, Last_Game_AST,\n  AverageAST, Last_Game_STL, AverageSTL, Last_Game_BLK, AverageBLK,\n  game_number, Home/Away, Opponent'

In [196]:
#Import the player file and do a few additions and checks
player = pd.read_csv('import.csv')
player_pos = player['Pos']
player_name = player['Name']
player = pd.get_dummies(player)
player = player.fillna(0)

In [197]:
original = pd.read_csv('F:\\Projects\\NBA\\DB_Files\\master.csv')
original.drop(columns=['PTS','Home', 'Away','Game', 'MIN', 'Date','FTM', '3P%', '3PA', 'PF', 'FG%', 'FGA',
       'OREB', 'FT%', 'FTA', 'DREB','+/-', '1QH', '2QH', '3QH', '4QH', '1QA', '2QA', '3QA',
       '4QA', 'Total_H', 'Total_A', 'W/L','Fantasy_Score',
        'TOV', 'REB', '3PM', 'FGM', 'AST', 'STL', 'BLK',], axis=1,inplace=True)
original = pd.get_dummies(original)
original = original.iloc[0:0]

In [198]:
combined = original.append(player)
combined = combined.fillna(0)


C:\Users\matth\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [199]:
results = pd.DataFrame()

In [200]:
targets = ['TOV', 'REB', '3PM', 'FGM', 'AST', 'STL', 'BLK']

In [201]:
for i in targets:
    model = joblib.load('F:\\Projects\\NBA\\Pickles\\' + i + '.pkl')
    predict = model.predict(combined)
    results[i] = predict
results['Name'] = player_name
results['POS'] = player_pos

C:\Users\matth\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.19.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\matth\Anaconda3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.19.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [202]:
def fantasy_points(df):
    score = (df['3PM'] * 3) + (df['FGM'] * 2) + (df['REB'] * 1) + (df['AST'] * 1.5) + (df['BLK'] * 3) + (df['STL'] * 3) + (df['TOV'] * -1)
    return score

results['Fantasy Score'] = results.apply(fantasy_points, axis=1)

## Salary Calc

In [203]:
salary = pd.read_csv('salary.csv')

In [204]:
salary.head()

,Name,Salary
0,Alan Williams,8476
1,Alex Len,9500
2,Anthony Tolliver,9947
3,Demarcus Green,7851
4,Antonius Cleveland,8310


In [205]:
salary_cap = 50000
team = ['PG', 'SG', 'SF', 'PF', 'C','PG', 'SG', 'SF', 'PF']

In [206]:
results['Salary'] = salary['Salary']
results['Value'] = salary['Salary']/results['Fantasy Score']

In [207]:
def pos_change(df_column):
    if df_column == 'F-C':
        return 'PF'
    elif df_column == 'G-F':
        return 'SG'
    elif df_column == 'F-C':
        return 'PF'
    elif df_column == 'F-G':
        return 'SF'
    elif df_column == 'G ':
        return 'PG'
    elif df_column == 'F':
        return 'PF'
    elif df_column == 'C-F':
        return 'C'
    else:
        return df_column

In [208]:
results['POS'] = results['POS'].apply(lambda x: pos_change(x))

In [209]:
results.POS.value_counts()

G     37
PF    27
C     16
SG     8
SF     6
PG     1
Name: POS, dtype: int64

In [210]:
results.head()

,TOV,REB,3PM,FGM,AST,STL,BLK,Name,POS,Fantasy Score,Salary,Value
0,1.6,6.3,2.5,2.7,2.0,0.3,0.5,Alan Williams,PF,23.00,8476,368.521739
1,3.2,7.2,1.1,1.9,2.2,0.4,0.7,Alex Len,C,17.70,9500,536.723164
2,3.2,2.3,0.8,1.7,1.0,0.3,0.7,Anthony Tolliver,PF,9.40,9947,1058.191489
3,0.4,1.7,0.4,5.8,1.0,0.3,0.6,Antonius Cleveland,G,18.30,7851,429.016393
4,5.7,4.7,3.1,2.3,2.7,1.2,0.7,Blake Griffin,PF,22.65,8310,366.887417


In [18]:
def CreateRandomTeam(df):    
    team = {
    'pg' : random.sample(set(df[df['POS'] == 'PG'].iloc[:,7]),1),
    'sg' : random.sample(set(df[df['POS'] == 'SG'].iloc[:,7]),1),
    'sf' : random.sample(set(df[df['POS'] == 'SF'].iloc[:,7]),1),
    'pf' : random.sample(set(df[df['POS'] == 'PF'].iloc[:,7]),1),
    'c' : random.sample(set(df[df['POS'] == 'C'].iloc[:,7]),1),
    }
        
    
    while True:
        pg = team['pg'][0]
        if pg in team['sg'] or pg in team['sf'] or pg in team['pf'] or pg in team['c']:
            team['pg'] = random.sample(pgs,1)
        else:
            break
    
    while True:
        sg = team['sg'][0]
        if sg in team['pg'] or sg in team['sf'] or sg in team['pf'] or sg in team['c']:
            team['sg'] = random.sample(sgs,1)
        else:
            break
    
    while True:
        sf = team['sf'][0]
        if sf in team['pg'] or sf in team['sg'] or sf in team['pf'] or sf in team['c']:
            team['sf'] = random.sample(sfs,1)
        else:
            break
    
    while True:
        pf = team['pf'][0]
        if pf in team['pg'] or pf in team['sg'] or pf in team['sf'] or pf in team['c']:
            team['pf'] = random.sample(pfs,1)
        else:
            break
    
    while True:
        c = team['c'][0]
        if c in team['pg'] or c in team['sg'] or c in team['sf'] or c in team['pf']:
            team['c'] = random.sample(cs,1)
        else:
            break
            
    return team

In [212]:
def GetTeamPointTotal(team):
    total_points = 0
    for i in team:
        name = team[i][0]
        points = results[results['Name'] == name].iloc[:,9]
        total_points += points.item()
    return total_points    

In [155]:
def GetTeamSalary(team):
    total_salary = 0
    for i in team:
        name = team[i][0]
        cost = salary[salary['Name'] == name].iloc[:,1]
        total_salary += cost.item()
    return total_salary

In [49]:
#def printTeam(team):
    #print(team['pg'][0])
    #print(team['sg'][0])
    #print(team['sf'][0])
    #print(team['pf'][0])
    #print(team['c'][0])
    

In [50]:
def CreatePopulation(count):
    return [CreateRandomTeam(results) for i in range(0,count)]

In [116]:
def fitness(team):
    points = GetTeamPointTotal(team)
    salary = GetTeamSalary(team)
    values = team.values()
    if salary > 50000:
        return 0
    return points

In [52]:
def grade(pop):
    'Find average fitness for a population.'
    summed = reduce(add, (fitness(team) for team in pop))
    return summed / (len(pop) * 1.0)

In [53]:
def listToTeam(players):
    return {
    'pg' : [players[0]],
    'sg' : [players[1]],
    'sf' : [players[2]],
    'pf' : [players[3]],
    'c' : [players[4]],
    }

In [54]:
def mutate(team):
    positions = ['pg','sg','sf','pf','c']
      
    random_pos = random.choice(positions)
    if random_pos == 'pg':
        team['pg'][0] = random.choice(pgs)
    if random_pos == 'sg':
        team['sg'][0] = random.choice(sgs)
    if random_pos == 'sf':
        team['sf'][0] = random.choice(sfs)
    if random_pos == 'pf':
        team['pf'][0] = random.choice(pfs)
    if random_pos == 'c':
        team['c'][0] = random.choice(cs)
        
    
    while True:
        pg = team['pg'][0]
        if pg in team['sg'] or pg in team['sf'] or pg in team['pf'] or pg in team['c']:
            team['pg'] = random.sample(pgs,1)
        else:
            break
    
    while True:
        sg = team['sg'][0]
        if sg in team['pg'] or sg in team['sf'] or sg in team['pf'] or sg in team['c']:
            team['sg'] = random.sample(sgs,1)
        else:
            break
    
    while True:
        sf = team['sf'][0]
        if sf in team['pg'] or sf in team['sg'] or sf in team['pf'] or sf in team['c']:
            team['sf'] = random.sample(sfs,1)
        else:
            break
    
    while True:
        pf = team['pf'][0]
        if pf in team['pg'] or pf in team['sg'] or pf in team['sf'] or pf in team['c']:
            team['pf'] = random.sample(pfs,1)
        else:
            break
    
    while True:
        c = team['c'][0]
        if c in team['pg'] or c in team['sg'] or c in team['sf'] or c in team['pf']:
            team['c'] = random.sample(cs,1)
        else:
            break
           
    return team

In [55]:
def evolve(pop, retain=0.55, random_select=0.1, mutate_chance=0.01):
    graded = [ (fitness(team), team) for team in pop]
    print(graded)
    graded = [ x[1] for x in sorted(graded, reverse=True)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]

    # randomly add other individuals to promote genetic diversity
    for individual in graded[retain_length:]:
        if random_select > random.random():
            parents.append(individual)

    # mutate some individuals
    for individual in parents:
        if mutate_chance > random.random():
            individual = mutate(individual)

    # crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = random.randint(0, parents_length-1)
        female = random.randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            babies = breed(male,female)
            for baby in babies:
                children.append(baby)
    parents.extend(children)
    return parents

In [214]:
best_teams=[]
valid_teams = []
history = []
p = CreatePopulation(15000)
#fitness_history = [grade(p)]
#fitness_history = []
#for i in range(40):
    #p = evolve(p)
    #fitness_history.append(grade(p))
    #valid_teams = [team for team in p if GetTeamSalary(team) <= 50000]
    #valid_teams = sorted(valid_teams, key=GetTeamPointTotal, reverse=True)
    #if len(valid_teams) > 0:
        #best_teams.append(valid_teams[0])
for i in p:
    teamSalary = GetTeamSalary(i)
    teamPoints = GetTeamPointTotal(i)
    i['Team Salary'] = teamSalary
    i['Team Points'] = teamPoints
    if teamSalary <= 50000:
        valid_teams.append(i)
valid_teams.sort(key=itemgetter('Team Points'), reverse=True)

TypeError: 'str' object is not callable